In [2]:
import pandas as pd
import transformers

## Reading dataset

In [ ]:
file_path= 'dataset_merged_iberlef.csv'
df= pd.read_csv(file_path)
# Filter by train dataset
train_df = df[df['source'] == 'train']
# Filter by dev dataset
dev_df = df[df['source'] == 'dev']

## Binary Classification

In [4]:
target_names= ['0','1']
class_names= ['Not Hope','Hope']

In [5]:
def map_label(row):
    if row['binary']=='Not Hope':
        return 0
    elif row['binary']=='Hope':
        return 1

train_df = train_df.assign(label= train_df.apply(map_label, axis=1))
dev_df = dev_df.assign(label= dev_df.apply(map_label, axis=1))

In [7]:
from simpletransformers.classification import ClassificationModel


from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
def run_ModelTransformer(transformermodel, train, test):
    # Create training data in the correct format for SimpleTransformers
    train_df = pd.DataFrame({
        'text': train['clean_text'],
        'labels': train['label']
    })

    # Ensure labels are integers (needed for SimpleTransformers)
    train_df['labels'] = train_df['labels'].astype(int)

    # Creating a DataFrame with just the text
    test_df = pd.DataFrame({
        'text': test['clean_text']
    })


    # Create a TransformerModel
    model = ClassificationModel(
        transformermodel[0],
        transformermodel[1],
        num_labels=2,
        use_cuda=True,
        ignore_mismatched_sizes=True,
        args={
            'num_train_epochs': 3,
            'learning_rate': 1e-5,
            'max_seq_length': 64,
            'use_multiprocessing': False,
            'use_multiprocessing_for_evaluation': False,
            'overwrite_output_dir': True
        }
    )

    # Train the model with the properly formatted DataFrame
    model.train_model(train_df)

    # For unlabeled test data, use predict method
    predictions, raw_outputs = model.predict(test_df['text'].tolist())

    # Save predictions to the original test DataFrame and export
    test['predicted'] = predictions
    test.to_csv('predictions_binary_iberlef.csv', index=None)

    # Output information about the prediction process
    print("Predictions completed")
    print(f"Predicted {len(predictions)} samples.")

    # Return the predictions
    return {
        'predictions': predictions
    }

In [ ]:
run_ModelTransformer(['auto', 'xlm-roberta-large'],train_df, dev_df)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Running Epoch 3 of 3:   0%|          | 0/2661 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Predictions completed for unlabeled test data.
Predicted 7491 samples.


{'predictions': array([1, 0, 0, ..., 1, 1, 1])}